In [13]:
!pip install -r requirements.txt

  Obtaining dependency information for boto3==1.29.2 from https://files.pythonhosted.org/packages/70/43/6f10af50c64e6184191a3fbda0889e9c9bc5c9224302cd41200968419e6d/boto3-1.29.2-py3-none-any.whl.metadata
  Using cached boto3-1.29.2-py3-none-any.whl.metadata (6.7 kB)
  Obtaining dependency information for botocore==1.32.2 from https://files.pythonhosted.org/packages/f4/a2/e03fbeaa13a25405ac6251d5deb10cd663416d7e45e46cf9e446a91b9e0c/botocore-1.32.2-py3-none-any.whl.metadata
  Using cached botocore-1.32.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached certifi-2023.5.7-py3-none-any.whl (156 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached cloudpickle-2.0.0-py3-none-any.whl (25 kB)
  Using cached config_box-0.2.1600119743-py3-none-any.whl (2.4 kB)
  Using cached cryptography-40.0.2-cp36-abi3-macosx_10_12_x86_64.whl (2.8 MB)
  Obtaining dependency information for deep-translator==1.11.4 from https://files.pythonhosted.org/packages/38/3f/61a8ef73236dbea83a1a063

In [15]:
import os
import pandas as pd
from snowflake.sqlalchemy import URL
import logging
import warnings
from pathlib import Path

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.INFO)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)
logging.basicConfig(level="INFO")
logger = logging.getLogger()
root = Path().cwd()

# add all the required files in this path
input_path = root / "inputs"
if not os.path.exists(input_path):
    os.makedirs(input_path, exist_ok=True)

# save all the files in this path
output_path = root / "outputs"
if not os.path.exists(output_path):
    os.makedirs(output_path, exist_ok=True)
logger.info(f"Reading dependencies from {input_path}...")
logger.info(f"saving files to {root}...")

ModuleNotFoundError: No module named 'snowflake.sqlalchemy'; 'snowflake' is not a package

In [14]:
!pip i

In [4]:
from sqlalchemy import create_engine


class extractorFromQuery:
    def __init__(self):
        self.credentials = {
            "account": "qfi-niro",
            "user": "niro_analytics",
            "password": "4qVe_V-aEq,j.i,",
            "warehouse": "NIRO_ANALYTICS_WAREHOUSE",
        }

    def queryExecutor(self, query: str, *args):
        engine = create_engine(URL(**self.credentials))
        res = pd.read_sql_query(query, engine, params=args)
        return res

In [ ]:
disbursed_cases = """
    select 
    la.id,
    ou.category,
    la.user_id,
    la.principal_amount,
    la.interest_rate,
    la.tenure,
    o.phone_number,
    o.offer_type,
    o.attributed_partner as "supply",
    ou.category as riskband,
    tu.income_source,
    tu.monthly_take_home_income,
    cpv.cpv_type,
    l.name as "demand",
    o.data_tag,
    CASE
        when la.loan_disbursed_date_by_ops is null then date(la.disbursement_date)
        else date(la.loan_disbursed_date_by_ops)
    end as disbursement_date
    from niro_rds.niro_data.core__public__loan_application la
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__OFFERS o on o.id = la.offer_id
    left join NIRO_RDS.NIRO_DATA.core__public__offer_user_details ou on ou.offer_id = o.id
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__VERIFICATION_DETAILS vd on vd.id = la.verification_id 
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__TELE_PD_USER tu on vd.telepd_id::bigint = tu.id
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__CP_VERIFICATION cpv on cpv.id = vd.cpv_id::bigint
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__VENDORS v on v.id = o.vendor_uid
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__LENDERS l on l.id = o.lender_uid
    where la.loan_application_status in ('CLOSED','LOAN_EXECUTED') and la.is_deleted = False and o.is_deleted = False;
"""
total_disbursed_cases = extractorFromQuery().queryExecutor(disbursed_cases)
total_disbursed_cases.shape

In [ ]:
disbursed_cases_with_cvattrs = """
            select * from (
            select 
            distinct la.id,
            pii.pin,
            pii.calculated_age,
            pii.city,
            tli.*,
            date_trunc("MONTH",date(od.created_at)) as createdate,
            od.current_salary,
            od.foir_dlq, od.foir_cibil, od.foir_trd, od.foir_mean, od.new_foir, 
            od.dec_reason,od.naps_score,
            rank()over(partition by od.user_id order by od.created_at asc) as rnk,
            CASE
                when la.loan_disbursed_date_by_ops is null then date(la.disbursement_date)
                else date(la.loan_disbursed_date_by_ops)
            END as disbursement_date
            from NIRO_RDS.NIRO_DATA.CORE__PUBLIC__LOAN_APPLICATION la
            left join NIRO_RDS.NIRO_DATA.core__public__offer_user_details ou on ou.offer_id = la.offer_id
            left join ANALYTICS.OFFERS.offer_details od on od.user_id = la.user_id and od.category = ou.category
            left join ANALYTICS.OFFERS.personal_info pii on od.pii_id = pii.pii_id
            left join ANALYTICS.OFFERS.tradeline_info tli on tli.tli_id = od.tli_id
            where la.loan_application_status in ('CLOSED','LOAN_EXECUTED') 
            and la.is_deleted = False
            and(la.is_deleted_at_source = false or la.is_deleted_at_source is null)
            and od.dec_reason = 'NOT DECLINED')
where rnk = 1;

"""
disbursed_cases = (
    extractorFromQuery()
    .queryExecutor(disbursed_cases_with_cvattrs)
    .drop(columns="disbursement_date")
)
disbursed_cases.shape

In [ ]:
disbursed_cases = disbursed_cases.drop_duplicates(subset=["id"])
final = total_disbursed_cases.merge(disbursed_cases, on="id", how="left").rename(
    columns={"id": "niro_opportunity_id"}
)
final.drop_duplicates(subset=["niro_opportunity_id"], inplace=True)
final.shape

In [ ]:
upper_cols = [
    "agg911",
    "rvlr01",
    "bcpmtstr",
    "cv11",
    "cv14",
    "mt28s",
    "mt33s",
    "pl33s",
    "at20s",
    "mt01s",
    "bc02s",
    "bg01s",
    "cv10",
    "trd",
    "at33a",
    "au33s",
    "co04s180",
    "au28s",
    "pl28s",
    "co01s180",
    "bc28s",
    "cv12",
    "co05s",
    "g310s",
    "aggs911",
    "at01s",
    "at33a_ne_ccod",
    "cv14_12m",
    "cv14_6m",
    "cv14_3m",
    "cv14_1m",
    "g310s_24m",
    "g310s_6m",
    "g310s_3m",
    "g310s_1m",
    "cv11_24m",
    "cv11_12m",
    "g057s_1dpd_36m",
    "g057s_1dpd_12m",
    "bc106s_60dpd",
    "bc107s_24m",
    "bc106s_60dpd_12m",
    "bc107s_12m",
    "bc106s_le_30dpd_12m",
    "bc09s_36m_hcsa_le_30",
    "pl09s_36m_hcsa_le_30",
    "at09s_6m",
    "g310s_36m",
    "at33a_ne_wo",
    "at09s_12m",
    "at09s_3m",
    "cv13",
    "cv24",
    "revs904",
    "cv20",
    "cv22",
    "ul_trd",
    "cv21",
    "g310s_2m",
]

cols = [
    "secured_accounts_count",
    "unsecured_accounts_count",
    "secured_high_credit_sum",
    "unsecured_high_credit_sum",
    "secured_amount_overdue_sum",
    "unsecured_amount_overdue_sum",
    "secured_balances_sum",
    "unsecured_balances_sum",
]

normal_cols = [
    "user_id",
    "niro_opportunity_id",
    "riskband",
    "pin",
    "city",
    "income_source",
    "monthly_take_home_income",
    "current_salary",
    "cpv_type",
    "principal_amount",
    "interest_rate",
    "tenure",
    "supply",
    "foir_dlq",
    "foir_cibil",
    "foir_trd",
    "foir_mean",
    "new_foir",
    "demand",
    "disbursement_date",
    "createdate",
    "dec_reason",
    "naps_score",
    "phone_number",
    "bureau_score",
    "calculated_age",
    "own_accounts_count",
    "other_accounts_count",
]

final = final[upper_cols + cols + normal_cols]
final.rename(columns={i: i.upper() for i in upper_cols}, inplace=True, errors="raise")
final.rename(
    columns={i: i.replace("_", " ").title() for i in cols}, inplace=True, errors="raise"
)
final.rename(
    columns={
        "phone_number": "Mobile",
        "bureau_score": "CIBILTUSC3 Score Value",
        "calculated_age": "Calculated_Age",
        "new_foir": "New_FOIR",
        "current_salary": "Current_Salary",
        "other_accounts_count": "Other Accounts count",
        "own_accounts_count": "Own Accounts count",
        "naps_score": "naps_tu",
        "pin": "pincode_Approved",
    },
    inplace=True,
    errors="raise",
)
final.head()

In [ ]:
final["disbursement_date_mod"] = final["disbursement_date"].astype(str).str[:7]
final.groupby("disbursement_date_mod").agg({"niro_opportunity_id": "count"})

In [ ]:
# basic ffill imputation
final.sort_values(by="riskband", inplace=True)
final.fillna(method="ffill", inplace=True)
final.shape

In [ ]:
# amounts validation w.r.t category
final[(final["disbursement_date_mod"] != "2023-12")].groupby("riskband").agg(
    {"principal_amount": "sum"}
)

In [ ]:
# Total amount validation
final["principal_amount"].sum()

In [ ]:
# saving the file
final.to_csv(output_path / "dataInputs_dec.csv")